## Regras de Associação

Regras de Associação identificam padrões comuns em itens de um grande conjunto de dados.Neste exercício, nós vamos analisar padrões de comportamento em uma plataforma de filmes (como o Netflix) onde as pessoas costumam assistir seus filmes e séries. Existem alguns padrões claros, como pessoas que gostam de super heróis ou aqueles que assistem a desenhos animados.

Regras de Associação são geralmente escritas no formato: **{A} -> {B}**,  o que siginifica que existe uma forte relação entre os itens A e B. Por exemplo, uma possível regra válida para a plataforma de streams é **{Senhor dos Anéis} -> {O Hobbit}**. 

Se frequentemente uma pessoa que assiste a um filme também assiste a um outro, ou seja os filmes são asssitidos frequentemente juntos, então a plataforma de filmes poderia utilizar esse padrão para aumentar a visualização de alguns filmes, através de recomendações na plataforma.

No exemplo acima, **{Senhor dos Anéis} -> {O Hobbit}**, {Senhor dos Anéis} é o **antecedente** e **{O Hobbit}** é o **consequente**. Antecedentes e consequentes podem ter múltiplos itens, por exemplo um regra válida é **{Thor: Ragnarok, Vingadores: Guerra Infinita}->{Vingadores: Ultimato}**.

Por quê?
Fácil de explicar para pessoas não-técnicas

Sem necessidade de grande preparação dos dados e engenharia de features

Bom início para explorar dados


## Identificando padrões frequentes em usuários de streaming de vídeos
Neste exemplo nós utilizaremos regras de associação para analisar um dataset de transações onde cada transação é composta pelos filmes que um mesmo usuário de uma plataforma de filmes assisitu dentro de um intervalo de tempo.

Exemplo baseado no tutorial disponível em: https://medium.com/@fabio.italiano/the-apriori-algorithm-in-python-expanding-thors-fan-base-501950d55be9

<img src="fig_apriori/Streaming-Movie.jpg">

### Passo 1) Leitura do dataset

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
df = pd.read_csv('dataset_movies/movie_dataset.txt',header=None)

In [ ]:
df.head()

Cada linha do arquivo refere-se a um conjunto de filmes que um determinado usuário leu. Vamos considerar esse conjunto de filmes como sendo o conjunto de itens de uma transação.

Entretanto, precisamos transforma os dados para deixá-lo num formato de um dataframe  onde cada coluna se refere a um filme e as linhas aos usuarios. Cada cálula contém 1 quando o usuário assitiu ao filme e 0 no caso contrário.

In [ ]:
import numpy as np

In [ ]:
rows = df.shape[0]

In [ ]:
filmes = set()
for i in range(rows):
    filmes = filmes.union(set(df.iloc[i].unique()))


In [ ]:
np.nan in filmes

In [ ]:
filmes.difference_update({np.nan})

In [ ]:
df_ = pd.DataFrame(columns=filmes,data=np.zeros((rows,len(filmes))))

In [ ]:
df_.head()

In [ ]:
def set_units(x):
    return 1

In [ ]:
for i in range(rows):
    df_.at[i, df.iloc[i].dropna()] = 1.

In [ ]:
df_.head()

### O Algoritmo Apriori
Alguns elementos são essenciais para o entendimento do algoritmo Apriori. 


**Suporte**: é um número de vezes que o itemset aparece em diferentes transações dividido pelo número total de transações.

$$supp(X) = \frac{|t \in T; X \subseteq t|}{|T|}$$

Por exemplo, podemos analisar o suporte do filme "Jumanji" fazendo a seguinte operação. 

In [ ]:
def supp(df_,X):
    union = np.prod(df_[X].values,axis=1)
    return len(np.nonzero(union)[0])/df_.shape[0]

In [ ]:
supp(df_,["Jumanji"])

In [ ]:
supp(df_,['Jumanji','Wonder Woman'])

**Itemset Frequente**: Um conjunto $\{i_1,i_2, ..., i_n\}$ de itens é frequente quando o conjunto de itens ocorre com pelo menos a frequênciade um supporte mínimo, $min\_supp$.

**Confiança**:é a indicação de quão frequente uma regra é verdadeira. Quanto maior a confiança, maior é chance de encontrarmos a regra no dataset. É dada por:

$$conf(X \rightarrow Y) = supp(X \cup Y)/supp(X)$$


Por exemplo, a confiança da regra **{Avengers} -> {Thor}** é dada por:

In [ ]:
def confidence(df_, X, Y):
    return supp(df_,X+Y)/supp(df_,X)

In [ ]:
confidence(df_, ['Avengers'], ['Thor'])

In [ ]:
confidence(df_, ['Thor'],  ['Avengers'])

In [ ]:
supp(df_, ['Avengers'])

In [ ]:
supp(df_, ['Thor'])

**Quando uma regra satisfaz a um mínimo suporte e confiança, dizemos que a regra é um regra de associação forte.**

Em geral, a mineração de regras de associação pode ser definida como:

1 - Encontrar todos os itemsets frequentes;

2 - Gerar regras de associação fortes a partir desses itens.

### Como funciona o algoritmo?

* Chamado de **Apriori** pois requer um conhecimento prévio das propriedades do itens mais frequentes;
* É um método iterativo onde $k$ itens são utilizados para para explorar $k+1$ itens;
* **Ideia geral**: Primeiro encontre o o itemset frequente de tamanho 1 satisfazendo o mínimo suporte, denominado $L_1$. Depois utilize $L_1$ para encontrar $L_2$, os itens frequentes de tamanho 2. $L_2$ é utilizado para encontrar $L_3$ e assim por diante.
* **Propriedade Apriori**: Todos os subconjuntos não vazios de um conjunto de itens frequente, também é frequente.




<img src="fig_apriori/Apriori.jpg">

Fonte: http://www.lessons2all.com/Apriori.php

### Utilizando o algortimo apriori

In [ ]:
frequent_itemsets = apriori(df_, min_support=0.01, use_colnames=True)

#### Visualizando itens frequentes

In [ ]:
frequent_itemsets.sort_values(by='support', ascending=False)

#### Computando regras de associação 

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)
rules.sort_values(by='confidence', ascending=False)

Suporte e confiança não são suficientes para filtrar regras interessantes. Uma medida de correlação também pode ser utilizada. O **lift** é uma medidade simples de correlação que mede se a corrência de um evento A é independente da ocorrência de um ecento B.

**Lift**: O lift de uma regra é definido como:  

$$lift(X \rightarrow Y): \frac{supp(X \cup Y)}{supp(X) \times supp(Y)}$$

* lift 1: a ocorrência de X é independente da ocorrência de Y

* lift > 1: possível dependência entre X e Y,  o que faz a regra útil para predizer futuros itens

* lift < 1: a presença X tem um efeito negativo na de Y, e vice-versa.


Por exemplo, a confiança da regra **{Avengers} -> {Thor}** é dada por:

In [ ]:
def lift(df_, X, Y):
    return supp(df_,X+Y)/(supp(df_,X)*supp(df_,Y))

#### Visualizando regras com determinada confiança e lift

In [ ]:
rules[ (rules['lift'] > 1.) &
       (rules['confidence'] >= 0.4) ]

In [ ]:
lift(df_,  ['Avengers'], ['Thor'])